In [14]:
import concurrent.futures
import pandas as pd
def parse_variables(lines):
    variables = {}
    for line in lines:
        if line.startswith("PROBLEM NAME:"):
            variables['problem_name'] = line.split(':')[1].strip()
        elif line.startswith("KNAPSACK DATA TYPE:"):
            variables['knapsack_data_type'] = line.split(':')[1].strip()
        elif line.startswith("DIMENSION:"):
            variables['dimension'] = int(line.split(':')[1].strip())
        elif line.startswith("NUMBER OF ITEMS:"):
            variables['num_items'] = int(line.split(':')[1].strip())
        elif line.startswith("CAPACITY OF KNAPSACK:"):
            variables['knapsack_capacity'] = int(line.split(':')[1].strip())
        elif line.startswith("MIN SPEED:"):
            variables['min_speed'] = float(line.split(':')[1].strip())
        elif line.startswith("MAX SPEED:"):
            variables['max_speed'] = float(line.split(':')[1].strip())
        elif line.startswith("RENTING RATIO:"):
            variables['renting_ratio'] = float(line.split(':')[1].strip())
        elif line.startswith("EDGE_WEIGHT_TYPE:"):
            variables['edge_weight_type'] = line.split(':')[1].strip()
    return variables

def parse_node_coord(lines, start_line, dimension):
    data = []
    for line in lines[start_line:start_line + dimension]:
        parts = line.split()
        index = int(parts[0])
        x = float(parts[1])
        y = float(parts[2])
        data.append({'Index': index, 'X': x, 'Y': y})
    return pd.DataFrame(data)

def parse_item_section(lines, start_line, dimension):
    data = []
    for line in lines[start_line:start_line + dimension]:
        index, profit, weight, assigned_node = map(int, line.split())
        data.append({'Index': index, 'Profit': profit, 'Weight': weight, 'Assigned_Node': assigned_node})
    return pd.DataFrame(data)

def parse_file_parallel(lines, dimension):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_node_coord = executor.submit(parse_node_coord, lines, 10, dimension)
        future_profit_matrix = executor.submit(parse_item_section, lines, 10 + dimension + 1, dimension)

    node_coord = future_node_coord.result()
    profit_matrix = future_profit_matrix.result()

    return node_coord, profit_matrix


# Example usage:

with open('test-example-n4.txt', 'r') as file:
    lines = file.readlines()

variables = parse_variables(lines)
node_coord, profit_matrix = parse_file_parallel(lines, variables['dimension'])

print("Variables:", variables)
print("Node Coord:", node_coord)
print("Profit Matrix:", profit_matrix)


Variables: {'problem_name': 'Test', 'knapsack_data_type': 'unknown', 'dimension': 4, 'num_items': 3, 'knapsack_capacity': 80, 'min_speed': 0.1, 'max_speed': 1.0, 'renting_ratio': 1.516, 'edge_weight_type': 'CEIL_2D'}
Node Coord:    Index    X    Y
0      1  0.0  0.0
1      2  4.0  0.0
2      3  8.0  3.0
3      4  0.0  3.0
Profit Matrix:    Index  Profit  Weight  Assigned_Node
0      2      34      30              2
1      3      40      40              3
2      4      25      21              4


In [15]:
variables

{'problem_name': 'Test',
 'knapsack_data_type': 'unknown',
 'dimension': 4,
 'num_items': 3,
 'knapsack_capacity': 80,
 'min_speed': 0.1,
 'max_speed': 1.0,
 'renting_ratio': 1.516,
 'edge_weight_type': 'CEIL_2D'}

In [16]:
node_coord

,Index,X,Y
0,1,0.0,0.0
1,2,4.0,0.0
2,3,8.0,3.0
3,4,0.0,3.0


In [17]:
profit_table=profit_matrix

In [18]:
import numpy as np

def calculate_distance_matrix(node_coord):
    coordinates = node_coord[['X', 'Y']].values
    x, y = np.meshgrid(coordinates[:, 0], coordinates[:, 1])

    distance_matrix = np.sqrt((x - x.T)**2 + (y - y.T)**2)
    
    return distance_matrix


In [19]:
distance_matrix=calculate_distance_matrix(node_coord)

In [20]:
distance_matrix

array([[0.        , 4.        , 8.54400375, 3.        ],
       [4.        , 0.        , 5.        , 5.        ],
       [8.54400375, 5.        , 0.        , 8.        ],
       [3.        , 5.        , 8.        , 0.        ]])

In [21]:
profit_table

,Index,Profit,Weight,Assigned_Node
0,2,34,30,2
1,3,40,40,3
2,4,25,21,4


In [26]:
packing_plan_example=[1,0,1]


In [27]:
profit_table['Picked'] = packing_plan_example


In [31]:
profit_table.to_clipboard()

In [22]:
pd.DataFrame(distance_matrix)

,0,1,2,3
0,0.000000,4.0,8.544004,3.0
1,4.000000,0.0,5.000000,5.0
2,8.544004,5.0,0.000000,8.0
3,3.000000,5.0,8.000000,0.0


In [23]:
pd.DataFrame(calculate_distance_matrix(node_coord))

,0,1,2,3
0,0.000000,4.0,8.544004,3.0
1,4.000000,0.0,5.000000,5.0
2,8.544004,5.0,0.000000,8.0
3,3.000000,5.0,8.000000,0.0


In [24]:
tour_example=[1,3,2,4]
packing_plan_example=[1,0,1]

In [80]:
variables

{'dimension': 4,
 'num_items': 3,
 'capacity': 80,
 'min_speed': 0.1,
 'max_speed': 1.0,
 'renting_ratio': 1.516,
 'edge_weight_type': 'CEIL_2D'}

In [88]:
def calculate_current_velocity(weight, capacity, min_speed, max_speed):
    if weight <= capacity:
        return max_speed - (weight / capacity) * (max_speed - min_speed)
    else:
        return min_speed


In [102]:
def calculate_cost(tour, packing_plan, variables,distance_matrix,profit_table):
    dimension = variables['dimension']
    max_speed = variables['max_speed']

    current_weight = 0
    total_time = 0
    current_velocity = max_speed

    for i in range(dimension):
        # Calculate the distance between consecutive cities
        distance = distance_matrix[tour[i % dimension]][tour[(i + 1) % dimension]]

        # Calculate the time taken to travel the distance
        time_to_travel = distance / current_velocity
        total_time += time_to_travel

        # Update current weight based on the packing plan
        current_weight += packing_plan[i % dimension] * profit_table['Weight'][i]

        # Calculate the current velocity
        current_velocity = calculate_current_velocity(current_weight, variables['capacity'],
                                                      variables['min_speed'], variables['max_speed'])

    return total_time


In [103]:
calculate_cost(tour=tour_example,packing_plan=packing_plan_example,variables=variables,distance_matrix=distance_matrix,profit_table=profit_table)

IndexError: index 4 is out of bounds for axis 0 with size 4

In [81]:
def calculate_cost(tour, packing_plan, min_speed, max_speed, renting_ratio, distance_matrix, profit_table):
        total_time = 0.0
        current_weight = 0

        # Initial velocity
        current_velocity = max_speed

        for i in range(len(tour)):
            current_city = tour[i]

            # Calculate the travel time to the next city
            if i < len(tour) - 1:
                next_city = tour[i + 1]
                travel_time = distance_matrix[current_city][next_city] / current_velocity
                total_time += travel_time

            # Update the knapsack weight and velocity based on packing plan
            if packing_plan[current_city - 1] == 1:
                current_weight += profit_table[current_city - 1]['Weight']
                current_velocity = max(min_speed, current_velocity - renting_ratio * current_weight)

        return total_time

In [82]:
variables

{'dimension': 4,
 'num_items': 3,
 'capacity': 80,
 'min_speed': 0.1,
 'max_speed': 1.0,
 'renting_ratio': 1.516,
 'edge_weight_type': 'CEIL_2D'}

In [85]:
calculate_cost(tour_example, packing_plan_example,
                                                variables['min_speed'],
                                                variables['max_speed'],
                                                variables['renting_ratio'],
                                                distance_matrix,
                                                profit_table)

KeyError: 0

In [ ]:
calculate_cost(tour_example, packing_plan_example,
                                                min_speed,
                                                max_speed,
                                                renting_ratio,
                                                distance_matrix,
                                                profit_table)

In [58]:
class TravellingThief:
    def __init__(self, population_size, dimension, num_items, capacity, min_speed, max_speed, renting_ratio, edge_weight_type, distance_matrix, profit_table):
        self.population_size = population_size
        self.dimension = dimension
        self.num_items = num_items
        self.capacity = capacity
        self.min_speed = min_speed
        self.max_speed = max_speed
        self.renting_ratio = renting_ratio
        self.edge_weight_type = edge_weight_type
        self.distance_matrix = distance_matrix
        self.profit_table = profit_table

    @staticmethod
    def calculate_cost(tour, packing_plan, min_speed, max_speed, renting_ratio, distance_matrix, profit_table):
        total_time = 0.0
        current_weight = 0

        # Initial velocity
        current_velocity = max_speed

        for i in range(len(tour)):
            current_city = tour[i]

            # Calculate the travel time to the next city
            if i < len(tour) - 1:
                next_city = tour[i + 1]
                travel_time = distance_matrix[current_city][next_city] / current_velocity
                total_time += travel_time

            # Update the knapsack weight and velocity based on packing plan
            if packing_plan[current_city - 1] == 1:
                current_weight += profit_table[current_city - 1]['Weight']
                current_velocity = max(min_speed, current_velocity - renting_ratio * current_weight)

        return total_time

# Example usage
travelling_thief_instance = TravellingThief(population_size=4, dimension=280, num_items=279, capacity=25936,
                                            min_speed=0.1, max_speed=1.0, renting_ratio=5.61, edge_weight_type='CEIL_2D',
                                            distance_matrix=distance_matrix, profit_table=profit_matrix)

tour_example = [1, 3, 2, 4]
packing_plan_example = [1, 0, 1]

cost = travelling_thief_instance.calculate_cost(tour_example, packing_plan_example,
                                                travelling_thief_instance.min_speed,
                                                travelling_thief_instance.max_speed,
                                                travelling_thief_instance.renting_ratio,
                                                travelling_thief_instance.distance_matrix,
                                                travelling_thief_instance.profit_table)

print(f"Cost for the tour and packing plan: {cost}")


KeyError: 0

In [1]:
import numpy as np

# Given vector
genotype_vector = [0.5, 0.1, 0.8, 0.6, 0.1, 0.9]

# Determine the length of the tour (n) and the length of the packing plan (m)
n = 4  # Assuming the tour has 4 cities
m = 3  # Assuming there are 3 items in the packing plan

# Split the genotype vector into two parts
tour_part = np.array(genotype_vector[:n-1])
packing_plan_part = np.array(genotype_vector[n-1:])

# Decode the tour part to a permutation vector
tour_permutation = np.argsort(tour_part) + 1
tour_permutation = np.insert(tour_permutation, 0, 1)  # Insert the starting city 1

# Decode the packing plan part to a binary decision vector
packing_decision_vector = (packing_plan_part > 0.5).astype(int)

# Display the results
print("Tour Phenotype:", tour_permutation)
print("Packing Plan Phenotype:", packing_decision_vector)


Tour Phenotype: [1 2 1 3]
Packing Plan Phenotype: [1 0 1]


In [2]:
        print(i,current_weight,current_velocity,distance,time_to_travel)


NameError: name 'variables' is not defined

In [13]:
packing_plan


NameError: name 'packing_plan' is not defined

In [32]:
profit_table

,Index,Profit,Weight,Assigned_Node,Picked
0,2,34,30,2,1
1,3,40,40,3,0
2,4,25,21,4,1


In [45]:
import numpy as np

n = 5  # Example value for n
vector_length = 2 * n - 1

unique_values = set()
while len(unique_values) < vector_length:
    value = np.random.uniform(low=0.0001, high=0.9999)  # Exclude 0 and 1
    unique_values.add(value)

vector = np.array(list(unique_values))


In [46]:
vector

array([0.88132891, 0.87708642, 0.91012356, 0.58740603, 0.82413326,
       0.91607612, 0.89317512, 0.91985402, 0.90248644])

In [47]:
[0.5, 0.1, 0.8, 0.6, 0.1, 0.9]

[0.5, 0.1, 0.8, 0.6, 0.1, 0.9]

In [53]:
sorted([0.5, 0.1, 0.8])

[0.1, 0.5, 0.8]

In [52]:
[x+1 for x in sorted([0.5, 0.1, 0.8])]

[1.1, 1.5, 1.8]

In [55]:
distance_matrix

array([[0.        , 4.        , 8.54400375, 3.        ],
       [4.        , 0.        , 5.        , 5.        ],
       [8.54400375, 5.        , 0.        , 8.        ],
       [3.        , 5.        , 8.        , 0.        ]])

In [64]:
import numpy as np

def get_tour(starting_index, vector):
    half_length = len(vector) // 2
    first_half = vector[:half_length]
    sorted_indices = np.argsort(first_half) + 2  # Adding 2 because of zero-indexing and to include the starting index
    sorted_indices = [starting_index] + sorted_indices.tolist()
    return sorted_indices

def get_packing_plan(vector):
    half_length = len(vector) // 2
    second_half = vector[half_length:]
    packing_plan = np.where(second_half > 0.5, 1, 0)
    return packing_plan

# Example vector
vector = np.array([0.5, 0.1, 0.8, 0.6, 0.1, 0.9])

# Get tour and packing plan
tour = get_tour(starting_index=1, vector=vector)
packing_plan = get_packing_plan(vector)

tour, packing_plan


([1, 3, 2, 4], array([1, 0, 1]))

In [71]:
string.split(" ")

['3.15645915e-01',
 '4.53030677e-01',
 '2.66982264e-01',
 '1.08928182e-01\n',
 '8.68166479e-01',
 '6.29728518e-01',
 '3.52518708e-01',
 '6.75375994e-02\n',
 '6.26350592e-01',
 '5.98660860e-01',
 '9.22172511e-02',
 '5.68515514e-02\n',
 '2.37132200e-01',
 '2.35403670e-01',
 '7.74371382e-01',
 '4.26011311e-02\n',
 '3.54695279e-01',
 '4.69166604e-01',
 '7.67579920e-01',
 '8.63516638e-01\n',
 '7.77620476e-01',
 '2.25900156e-01',
 '|7.60389215e-01',
 '4.41385371e-01\n',
 '2.17436735e-01',
 '6.20251314e-01',
 '1.41223510e-01',
 '8.87551809e-02\n',
 '4.37561248e-01',
 '2.89188192e-01',
 '2.73069231e-01',
 '5.49044754e-01\n',
 '9.65158649e-01',
 '6.92406344e-01',
 '6.44791747e-01',
 '7.25829272e-01\n',
 '9.38300100e-01',
 '4.72944714e-02',
 '2.19055251e-03',
 '3.46374663e-01\n',
 '6.77973870e-01',
 '3.14115205e-01',
 '7.83066568e-01',
 '6.05550374e-01\n',
 '1.12060886e-01',
 '8.86877968e-01',
 '3.42211267e-01',
 '5.51175913e-01\n',
 '8.15684753e-01',
 '8.54537963e-01',
 '6.27288426e-01',
 '1.49

In [72]:
li=[0.3156459145887941, 0.45303067684589227, 0.26698226388996205, 0.10892818203683619, 0.8681664792853785, 0.6297285178269824, 0.3525187079173151, 0.06753759936467085, 0.6263505918617513, 0.598660859897128, 0.09221725110274981, 0.0568515513793274, 0.23713219972813737, 0.23540367038816268, 0.7743713820689289, 0.042601131064823106, 0.3546952789784885, 0.4691666036213711, 0.7675799195076131, 0.8635166378141325, 0.7776204763382165, 0.22590015600345126, 0.7603892152202866, 0.44138537149952684, 0.21743673513735529, 0.62025131416295, 0.14122350997088706, 0.08875518094267154, 0.4375612479314387, 0.2891881918604198, 0.27306923099739255, 0.5490447535955015, 0.9651586488653711, 0.6924063442845048, 0.6447917473135104, 0.7258292717802757, 0.938300100436643, 0.04729447143597154, 0.00219055250659117, 0.34637466323565347, 0.6779738699495301, 0.31411520478102106, 0.7830665680266904, 0.605550373785455, 0.11206088569114359, 0.8868779679014628, 0.34221126715587613, 0.5511759133297621, 0.8156847528063099, 0.8545379633556507, 0.6272884263049889, 0.14949659564350393, 0.7118526429170441, 0.3207274555339743, 0.5433641471356299, 0.5897104068982717, 0.2316469169551113, 0.01132707703427771, 0.7283014352571209, 0.051145034782327015, 0.8181671247290361, 0.24574870964620765, 0.07612764781499481, 0.5957480279907474, 0.18117441380363464, 0.21668067599417307, 0.5121412977133879, 0.0504229606830916, 0.2567363995936205, 0.25445189107853716, 0.3581139428419787, 0.5040175402808805, 0.15192437854690732, 0.5807286961807341, 0.1892970179207829, 0.33120903502442467, 0.3924006364099758, 0.4223238960915662, 0.40588680810555433, 0.5841893561610041, 0.9916614427498406, 0.13450459137706383, 0.5070360609515154, 0.8089085299731219, 0.964719688482097, 0.794197164693605, 0.17861191362207218, 0.08669080314213873, 0.8244824216067248, 0.26064404093048654, 0.9290106027277159, 0.9665951585828803, 0.1895053519009181, 0.9642229908458645, 0.648914840647448, 0.23861518259370162, 0.8013239803179384, 0.7660658506233377, 0.21111868954765878, 0.0955127134632553, 0.6950517148251113, 0.43815164779876403, 0.5786236268072228, 0.7164337361475963, 0.352323403845057, 0.3088845044054649, 0.06767753715001135, 0.41877112897436364, 0.23036009500715837, 0.30612098059008774, 0.4040728855609139, 0.9391644402518545, 0.18069646361870595, 0.1944420893565616, 0.4561412810924149, 0.9170056527540128, 0.10816042084360566, 0.3876383233853633, 0.07983007673511666, 0.3985515880265039, 0.8758824607167398, 0.3320689286386461, 0.6559761575003882, 0.9387860047861156, 0.17342774190247046, 0.6113024471352992, 0.06375632254877317, 0.12460412449167146, 0.41511351695097853, 0.14886651460388767, 0.8859455327574751, 0.5901066878969492, 0.6828448534386378, 0.1007241544693005, 0.8934092393382715, 0.36648487602843105, 0.8496096752160812, 0.7056592386414977, 0.0012905095025858015, 0.6053780131654278, 0.0213653216052051, 0.41410442023456495, 0.049117023369169455, 0.9922620170055846, 0.04689654681105282, 0.1417398697692488, 0.25197843843179957, 0.3293587881178853, 0.7448832212939456, 0.19235949995070134, 0.8517108546254921, 0.47803830112981316, 0.9902589425525538, 0.3474595759134068, 0.5367936307385495, 0.22079193265818087, 0.645009298376699, 0.6421825981846332, 0.7970795059190872, 0.5698675266943178, 0.34001406013575186, 0.9867909667476272, 0.5898098693250206, 0.850221710716835, 0.3163178740581536, 0.8125100678449462, 0.2843373519411725, 0.6612817146697997, 0.11497708700706888, 0.9801588098481688, 0.8010162050937185, 0.9290124848181095, 0.7344054791286664, 0.3145975712712741, 0.9985589271843139, 0.5163426073327297, 0.3453456708404369, 0.8110410514684474, 0.286532990688805, 0.2009826126829134, 0.31412668502227337, 0.4004258388669161, 0.6519254805310977, 0.030331208469541404, 0.18784176093567184, 0.18284166060510576, 0.676988918635544, 0.531352267135199, 0.17521560924541846, 0.45856430531562054, 0.7869875685868247, 0.0372696089979172, 0.10825858203106509, 0.2916320333166027, 0.8529080693423841, 0.683483127335706, 0.012065449277768203, 0.7757313057710071, 0.8148503366211726, 0.9588015645345631, 0.9443860619287867, 0.748724587173887, 0.8848563686513269, 0.28287273121823553, 0.9922400792559454, 0.5992058857264638, 0.22833342017790115, 0.80376888006334, 0.04631912066615351, 0.6267813990970771, 0.9830727177406352, 0.36267813824000994, 0.6088376493488366, 0.7641263952563409, 0.509902682836008, 0.9359386259146053, 0.21091750065732862, 0.906093719759363, 0.9499013068440738, 0.8971815790081187, 0.47733508100609234, 0.7314830751088555, 0.09327386910244251, 0.4401951968630018, 0.15154513101297074, 0.10491616207776355, 0.5961205733041371, 0.49854811880381245, 0.6640628204310525, 0.23959031467982583, 0.21671648161217527, 0.14173621378819468, 0.45650572434948067, 0.08911015389412091, 0.6675905934560077, 0.8029634203589886, 0.3391774437371089, 0.09623768684077294, 0.8283635764962742, 0.5133358965763406, 0.49075056558053465, 0.596443244345152, 0.46560462528404667, 0.898784391748118, 0.8112551239804714, 0.9287122080352603, 0.4624985521570233, 0.6074206625124279, 0.019089965472385506, 0.4919515895936175, 0.7448337209347418, 0.921511532470731, 0.12050238852073636, 0.5357350458660195, 0.2767400379328825, 0.775236759353655, 0.29717542420915644, 0.703760605164089, 0.7891098518280023, 0.345365903550494, 0.5428180512043554, 0.9434202131134207, 0.7902629026315791, 0.6379420227671971, 0.09902479244233042, 0.2619892562236382, 0.5590681209893089, 0.26656416375065306, 0.48550595904856486, 0.7473906727826023, 0.04859748099627148, 0.9574503843573655, 0.7523901256683023, 0.9320107669189152, 0.722470775865361, 0.6333828905627944, 0.7316231341615282, 0.9235392067990574, 0.2659500224244138, 0.72523492888277, 0.8005072437434453, 0.05725185134540878, 0.0021387725667302293, 0.46144929572333016, 0.5217966168126003, 0.6589158554595917, 0.8308460164940072, 0.29739418417650065, 0.9349860897746808, 0.6288095099875372, 0.5579893063143778, 0.12100222119370008, 0.8492395507956426, 0.08298549168695513, 0.090529250158218, 0.6227718332228445, 0.4946374742846199, 0.8240572961128013, 0.11397209760031068, 0.5569952163305171, 0.28343798075049165, 0.6419141468035628, 0.644857492076062, 0.8704564796997485, 0.7824063848296426, 0.14770498291546919, 0.03814844916838278, 0.5139789214194819, 0.11872634712639984, 0.6187546627043877, 0.3701161270514437, 0.18595011097483494, 0.1500075154097804, 0.06334924595456337, 0.20565070189145884, 0.2719881738284662, 0.7227157777192468, 0.6045679666028697, 0.32518665944982106, 0.5932334843083672, 0.39047660729354583, 0.12719427675345818, 0.3379887107661339, 0.16554980493100324, 0.19615188937183825, 0.2932661178347783, 0.2883825538074124, 0.3296075345227515, 0.5941749164128975, 0.645579766268141, 0.12145303055119394, 0.7389137690750219, 0.8293608271282386, 0.9479095967233254, 0.7111884064275658, 0.7520128771543659, 0.012531506298104844, 0.9131035157438179, 0.47497729371319897, 0.021232836636396746, 0.516698078469757, 0.9614473790072655, 0.5656713587767103, 0.8437540134583638, 0.44771837771181777, 0.5776105879643083, 0.06920649257710731, 0.1599114158786209, 0.4859920257914109, 0.06676880814857378, 0.4579018848545644, 0.5268998991219587, 0.42523938785043003, 0.33220786338592667, 0.23741789139152247, 0.9641394547596961, 0.4566453208722051, 0.30802589912829503, 0.706525092695286, 0.6768473766847605, 0.014478462760402988, 0.053885247059237584, 0.3364820257005814, 0.5491020695486243, 0.2781512374688023, 0.3743928529129724, 0.8079302132266353, 0.9498996666913966, 0.2674153375122398, 0.3259248234822115, 0.36810636592708645, 0.7612641080949679, 0.6527603649059606, 0.521900179323565, 0.27926146816739406, 0.7408480632165825, 0.4313654686336137, 0.7779815670463393, 0.7352193569588182, 0.224699268264699, 0.5612898342041709, 0.7313261794943725, 0.045386433931651204, 0.23705585671415919, 0.27801947868310783, 0.9831523952580428, 0.723890479467354, 0.8683150622244388, 0.6938673968602254, 0.39897509282966914, 0.43902924476827376, 0.47930253333809547, 0.5634368292716468, 0.9206897958321371, 0.2790475558834238, 0.0009394319451263566, 0.24627574904488014, 0.8115721878452756, 0.3338445384605033, 0.3260861768548793, 0.8478592788014264, 0.07402465729600949, 0.0455168324391233, 0.10219888051561132, 0.2293641756088448, 0.8727134173409363, 0.5458531580395906, 0.6729291911538127, 0.6351901481492471, 0.9921569021636727, 0.5734379432749245, 0.47350765997840927, 0.3869373807775044, 0.6200418533344576, 0.6557256550637021, 0.5452958397630658, 0.37320567089444234, 0.7914175083063099, 0.36666491452024697, 0.019109281453525973, 0.20676215820595323, 0.2155809397628322, 0.5345113979555056, 0.8149556675425441, 0.10349985331164013, 0.5610933301538177, 0.5360747299965319, 0.6925257680159849, 0.23825116276536595, 0.7152818592688028, 0.6877952517882183, 0.9136450989333341, 0.6400451208787128, 0.8279269795985392, 0.1308632538226031, 0.7221876528328622, 0.4821513364326363, 0.7651305872708754, 0.7185538548744576, 0.05054649690664503, 0.09039138064473251, 0.6885582806756956, 0.9254353811690882, 0.3286669239334927, 0.629965575714349, 0.7685117856712026, 0.4310660530175925, 0.2830207031328029, 0.5073960357188914, 0.21323705871108034, 0.4285935007206376, 0.3364041399432254, 0.6898774559131599, 0.9372699716848855, 0.8472300160191893, 0.006775924907119757, 0.390033395616192, 0.385674697929584, 0.1622084608073896, 0.19152736337950094, 0.44946281604802185, 0.11681218717047326, 0.8895361505279618, 0.025696306825996218, 0.03352332270439362, 0.8894569203568178, 0.5011507485398705, 0.4366716712178933, 0.6228091336868652, 0.5004266240776057, 0.4254587621303989, 0.181275554230669, 0.6172525712544509, 0.17295883392635847, 0.35315665432007093, 0.30142559151270576, 0.8242948055277927, 0.10629061235059933, 0.688192624154925, 0.11884291768238897, 0.37781622034716444, 0.9331517097166417, 0.8457084922839767, 0.6103734868087306, 0.9637666484691381, 0.6006005670760887, 0.5451466699827286, 0.3265081577008242, 0.630131409740952, 0.39911671072710275, 0.9424215443838382, 0.7298006944233189, 0.7241921403716223, 0.7488980897223277, 0.42936205757782353, 0.7075202368490765, 0.20393033880969258, 0.2675443051267543, 0.05724910655780602, 0.034773334746559414, 0.0004929599290637299, 0.7501344293205403, 0.5762713671288953, 0.5834356656275739, 0.5938696796656547, 0.7237255286412801, 0.06293770097848683, 0.7051688153327156, 0.5595701410752102, 0.6066215680938344, 0.8731483100519596, 0.11447124343969106, 0.613515504463407, 0.517464266282973, 0.19217172400356675, 0.8645699160489357, 0.639381666314003, 0.9289928322829022, 0.04652403746849698, 0.5730592030419683, 0.6960252359472803, 0.22400093583803027, 0.40886952590640535, 0.6974225906984293, 0.031109300787134586, 0.6092083736812571, 0.8073107071092475, 0.6144456660710494, 0.49719014737545464, 0.013161696969343217, 0.9485813748104505, 0.8824297597731261, 0.8923071191102329, 0.946584796485818, 0.5625278951185478, 0.04389993025446448, 0.5479128478578591, 0.790657419742611, 0.4409068825380571, 0.0969686196927706, 0.18760366780225002, 0.9467991077722898, 0.6283772839310511, 0.9320804430906787, 0.9252290358222348, 0.21171871365820516, 0.6710159667297656, 0.8770043411790662, 0.48070250648438706, 0.8927619130524862, 0.4059825709255137, 0.15873119753151843, 0.8613798836168728, 0.8354154110895835, 0.25870699655606855, 0.31232471633380776, 0.028128901526816463, 0.6835575371315897]

In [73]:
len(li)

558

In [74]:
vector=np.array(li)

# Get tour and packing plan
tour = get_tour(starting_index=1, vector=vector)
packing_plan = get_packing_plan(vector)

tour, packing_plan


([1,
  140,
  40,
  59,
  198,
  250,
  142,
  185,
  193,
  17,
  210,
  146,
  39,
  272,
  144,
  69,
  61,
  13,
  128,
  9,
  108,
  64,
  120,
  89,
  29,
  235,
  12,
  224,
  101,
  239,
  266,
  135,
  227,
  118,
  194,
  5,
  46,
  170,
  254,
  129,
  83,
  28,
  233,
  147,
  131,
  53,
  226,
  74,
  126,
  190,
  88,
  114,
  66,
  187,
  186,
  76,
  94,
  151,
  115,
  181,
  218,
  100,
  67,
  232,
  26,
  157,
  23,
  208,
  110,
  58,
  15,
  14,
  97,
  231,
  63,
  148,
  71,
  70,
  91,
  267,
  280,
  269,
  4,
  32,
  256,
  205,
  168,
  180,
  31,
  195,
  258,
  111,
  107,
  43,
  182,
  175,
  2,
  166,
  55,
  149,
  77,
  123,
  238,
  162,
  48,
  178,
  261,
  41,
  155,
  106,
  8,
  18,
  72,
  213,
  137,
  119,
  78,
  121,
  183,
  112,
  80,
  143,
  130,
  109,
  79,
  30,
  103,
  225,
  25,
  3,
  116,
  234,
  191,
  248,
  244,
  19,
  222,
  153,
  270,
  242,
  251,
  229,
  73,
  84,
  216,
  68,
  241,
  177,
  189,
  255,
  156,
  262,

In [76]:
sorted(tour)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [1]:
import concurrent.futures
import pandas as pd
import numpy as np

def parse_variables(lines):
    variables = {}
    for line in lines:
        if line.startswith("PROBLEM NAME:"):
            variables['problem_name'] = line.split(':')[1].strip()
        elif line.startswith("KNAPSACK DATA TYPE:"):
            variables['knapsack_data_type'] = line.split(':')[1].strip()
        elif line.startswith("DIMENSION:"):
            variables['dimension'] = int(line.split(':')[1].strip())
        elif line.startswith("NUMBER OF ITEMS:"):
            variables['num_items'] = int(line.split(':')[1].strip())
        elif line.startswith("CAPACITY OF KNAPSACK:"):
            variables['knapsack_capacity'] = int(line.split(':')[1].strip())
        elif line.startswith("MIN SPEED:"):
            variables['min_speed'] = float(line.split(':')[1].strip())
        elif line.startswith("MAX SPEED:"):
            variables['max_speed'] = float(line.split(':')[1].strip())
        elif line.startswith("RENTING RATIO:"):
            variables['renting_ratio'] = float(line.split(':')[1].strip())
        elif line.startswith("EDGE_WEIGHT_TYPE:"):
            variables['edge_weight_type'] = line.split(':')[1].strip()
    return variables
def parse_node_coord(lines, start_line, dimension):
    data = []
    for line in lines[start_line:start_line + dimension]:
        parts = line.split()
        index = int(parts[0])
        x = float(parts[1])
        y = float(parts[2])
        data.append({'Index': index, 'X': x, 'Y': y})
    return pd.DataFrame(data)

def parse_item_section(lines, start_line, dimension):
    data = []
    for line in lines[start_line:start_line + dimension]:
        index, profit, weight, assigned_node = map(int, line.split())
        data.append({'Index': index, 'Profit': profit, 'Weight': weight, 'Assigned_Node': assigned_node})
    return pd.DataFrame(data)

def parse_file_parallel(lines, dimension):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_node_coord = executor.submit(parse_node_coord, lines, 10, dimension)
        future_profit_matrix = executor.submit(parse_item_section, lines, 10 + dimension + 1, dimension)

    node_coord = future_node_coord.result()
    profit_matrix = future_profit_matrix.result()

    return node_coord, profit_matrix


# Example usage:

with open('/Users/devarshigoswami/Desktop/work/nature_inspired_computation/TravellingThief/gecco19-thief/ECMM409_CA___Group_Project/test-example-n4.txt', 'r') as file:
    lines = file.readlines()

variables = parse_variables(lines)
node_coord, profit_matrix = parse_file_parallel(lines, variables['dimension'])

print("Variables:", variables)
print("Node Coord:", node_coord)
print("Profit Matrix:", profit_matrix)

profit_table=profit_matrix.copy()

def calculate_distance_matrix(node_coord):
    coordinates = node_coord[['X', 'Y']].values
    x, y = np.meshgrid(coordinates[:, 0], coordinates[:, 1])

    distance_matrix = np.sqrt((x - x.T)**2 + (y - y.T)**2)

    return distance_matrix

distance_matrix=calculate_distance_matrix(node_coord)

tour_example=[1,3,2,4]
packing_plan_example=[1,0,1]
profit_table['Picked'] = packing_plan_example


def calculate_current_velocity(weight, capacity, min_speed, max_speed):
    if weight <= capacity:
        return max_speed - (weight / capacity) * (max_speed - min_speed)
    else:
        return min_speed

# def calculate_cost(tour, packing_plan, variables,distance_matrix,profit_table):
#     dimension = variables['dimension']
#     max_speed = variables['max_speed']

#     current_weight = 0
#     total_time = 0
#     current_velocity = max_speed

#     for i in range(dimension):
#         # Calculate the distance between consecutive cities
#         distance = distance_matrix[tour[(i % dimension)]-1][tour[(i + 1) % dimension]-1]

#         # Calculate the time taken to travel the distance
#         time_to_travel = distance / current_velocity
#         total_time += time_to_travel

#         # Update current weight based on the packing plan
#         current_weight += (profit_table[profit_table['Assigned_Node']==tour[i+1]]['Weight']*profit_table[profit_table['Assigned_Node']==tour[i+1]]['Picked']).values[0]
#         # Calculate the current velocity
#         current_velocity = calculate_current_velocity(current_weight, variables['knapsack_capacity'],variables['min_speed'], variables['max_speed'])
#         print(i,current_weight,current_velocity,distance,time_to_travel)

#     return total_time

# print(calculate_cost(tour=tour_example,packing_plan=packing_plan_example,variables=variables,distance_matrix=distance_matrix,profit_table=profit_table))


def calculate_cost(tour, packing_plan, variables, distance_matrix, profit_table):
    dimension = variables['dimension']
    max_speed = variables['max_speed']

    total_time = 0
    current_velocity = max_speed
    current_weight = 0

    for i in range(dimension):
        # Calculate the distance between consecutive cities
        from_city = tour[i]
        to_city = tour[(i + 1) % dimension]
        distance = distance_matrix[from_city - 1][to_city - 1]

        # Calculate the time taken to travel the distance
        time_to_travel = distance / current_velocity

        # Explain the current step
        print(f"At city {from_city}:")
        print(f"  Knapsack weight before picking: {current_weight}")
        print(f"  Current velocity: {current_velocity}")
        print(f"  Distance to next city ({to_city}): {distance}")
        print(f"  Time to travel: {time_to_travel}")

        if i < len(packing_plan):
            # Update current weight based on the packing plan
            picked_item_weight = (profit_table[(profit_table['Assigned_Node'] == to_city)]['Weight']*profit_table[(profit_table['Assigned_Node'] == to_city)]['Picked']).values[0]
            current_weight += picked_item_weight * packing_plan[i]
            print(f"  Knapsack weight after picking item {i + 1}: {current_weight}")

        # Calculate the current velocity
        current_velocity = calculate_current_velocity(current_weight, variables['knapsack_capacity'], variables['min_speed'], variables['max_speed'])
        print(f"  Updated velocity: {current_velocity}")

        # Accumulate the time taken for this step
        total_time += time_to_travel

    # Calculate the time taken to return from the last city to the first city
    total_time += distance_matrix[tour[-1] - 1][tour[0] - 1] / current_velocity

    print(f"Total traveling time: {total_time}")
    return total_time


# Example usage:
print(calculate_cost(tour=tour_example, packing_plan=packing_plan_example, variables=variables, distance_matrix=distance_matrix, profit_table=profit_table))



Variables: {'problem_name': 'Test', 'knapsack_data_type': 'unknown', 'dimension': 4, 'num_items': 3, 'knapsack_capacity': 80, 'min_speed': 0.1, 'max_speed': 1.0, 'renting_ratio': 1.516, 'edge_weight_type': 'CEIL_2D'}
Node Coord:    Index    X    Y
0      1  0.0  0.0
1      2  4.0  0.0
2      3  8.0  3.0
3      4  0.0  3.0
Profit Matrix:    Index  Profit  Weight  Assigned_Node
0      2      34      30              2
1      3      40      40              3
2      4      25      21              4
At city 1:
  Knapsack weight before picking: 0
  Current velocity: 1.0
  Distance to next city (3): 8.54400374531753
  Time to travel: 8.54400374531753
  Knapsack weight after picking item 1: 0
  Updated velocity: 1.0
At city 3:
  Knapsack weight before picking: 0
  Current velocity: 1.0
  Distance to next city (2): 5.0
  Time to travel: 5.0
  Knapsack weight after picking item 2: 0
  Updated velocity: 1.0
At city 2:
  Knapsack weight before picking: 0
  Current velocity: 1.0
  Distance to next c

In [78]:
print(calculate_cost(tour=tour_example, packing_plan=packing_plan_example, variables=variables, distance_matrix=distance_matrix, profit_table=profit_table))


At city 1:
  Knapsack weight before picking: 0
  Current velocity: 1.0
  Distance to next city (3): 8.54400374531753
  Time to travel: 8.54400374531753
  Knapsack weight after picking item 1: 0
  Updated velocity: 1.0
At city 3:
  Knapsack weight before picking: 0
  Current velocity: 1.0
  Distance to next city (2): 5.0
  Time to travel: 5.0
  Knapsack weight after picking item 2: 0
  Updated velocity: 1.0
At city 2:
  Knapsack weight before picking: 0
  Current velocity: 1.0
  Distance to next city (4): 5.0
  Time to travel: 5.0
  Knapsack weight after picking item 3: 21
  Updated velocity: 0.7637499999999999
At city 4:
  Knapsack weight before picking: 21
  Current velocity: 0.7637499999999999
  Distance to next city (1): 3.0
  Time to travel: 3.9279869067103115
  Updated velocity: 0.7637499999999999
Total traveling time: 26.399977558738158
26.399977558738158
